In [1]:
import requests
from bs4 import BeautifulSoup
import pinecone
from dotenv import load_dotenv
load_dotenv()
pinecone.init(api_key="39e7f82b-83bf-4797-9281-0b76cb1e5b56", environment="us-west4-gcp-free")
import openai
index = pinecone.Index("nychackathon")
import numpy as np
import openai
import json
import os
open_ai_api_key = os.getenv("OPENAI_API_KEY")


/Users/noahkasmanoff/anaconda3/envs/nasaeo/lib/python3.9/site-packages/pinecone/index.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:
def get_openai_embedding(text):
    api_url = "https://api.openai.com/v1/embeddings"
    headers = {
        "Authorization": f"Bearer {open_ai_api_key}",
        "Content-Type": "application/json"
    }
    data = {
        "input": text,
        "model": "text-embedding-ada-002",
        "encoding_format": "float"
    }
    try:
        response = requests.post(api_url, headers=headers, json=data)
        if response.status_code == 200:
            embedding = json.loads(response.text)["data"][0]["embedding"]
            return embedding
        else:
            print(f"Failed to get embedding: {response.text}")
            return None
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

In [17]:
question = 'tengo hambre por favor dame comida?'
query = get_openai_embedding(question)
results = index.query(queries=[query], top_k=3, include_metadata=True, include_values=False)
matched_articles = results['results'][0]['matches']


In [16]:
matched_articles[0]

{'id': 'https://portal.311.nyc.gov/article/?kanumber=KA-03304',
 'metadata': {'page_text': '\n'
                           '\n'
                           '\n'
                           '\n'
                           '\n'
                           '\n'
                           '\r\n'
                           '\t\r\n'
                           '\t\t\r\n'
                           '\t\t\r'
                           '\r'
                           '\r'
                           '\r'
                           '\r'
                           '\r'
                           '\r'
                           '\r'
                           '\r'
                           '  \r'
                           'Food Resources\xa0· NYC311\r\n'
                           '\n'
                           '\n'
                           '\n'
                           '\n'
                           '\n'
                           '\n'
                           '\n'
                          

In [10]:
matched_articles = results['results'][0]['matches']

In [11]:
page_texts = []
websites = []
for article in matched_articles:
    website = article['id']
    page_text = article['metadata']['page_text']
    page_texts.append(page_text)
    websites.append(website)


ApiAttributeError: ScoredVector has no attribute 'metadata' at ['['received_data', 'results', 0, 'matches', 2]']['metadata']

In [ ]:
combined_contexts = "\n\n---\n\n".join(page_texts)

# build our prompt with the retrieved contexts included
prompt_start = (
    "You are the NYC AI Assistant. Please provide a short summary of the context below to answer the subsequent question. If the question is not relevant, politely ask the user to clarify.\n\n"+
    "Context:\n"
)
prompt_end = (
    f"\n\nQuestion: {question}\nAnswer:"
)

prompt = prompt_start + combined_contexts + prompt_end


In [ ]:
prompt = prompt.replace('\n', ' ')
# replace multiple whitespace with single whitespace
prompt = ' '.join(prompt.split())


In [ ]:
def complete(prompt):
    response = openai.ChatCompletion.create(
        model='gpt-3.5-turbo',
        messages=[
            {'role': 'user', 'content': prompt}
        ],
        temperature=0,
        max_tokens=550,
        
    )
    return response['choices'][0]['message']['content']

In [ ]:
answer = complete(prompt)

In [ ]:
sources_help = "Here are some websites you may find useful for more info: \n- " + "\n- ".join(list(set(websites)))   

In [ ]:
print(prompt)

In [ ]:
print(answer)

In [ ]:
print(sources_help)